### Setup

In [56]:
import os
import dotenv
import json
from pydantic import BaseModel, Field, validator
from langchain import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from jobgpt.resume_analyzer.resume_reader import ResumeReader
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import SequentialChain
import langchain
from jobgpt.utils.llm import count_tokens
from typing import List
dotenv.load_dotenv(".env", override=True)

True

In [9]:
langchain.debug = True

In [2]:
class ResumeAnalysis(BaseModel):
    evaluation: str = Field(description="The overall evaluation of the resume")
    suggestions: List[str] = Field(description="Suggestions for the resume, format into list")
    revised_resume: str = Field(description="The revised resume")

In [3]:
# initialize the models
openai = OpenAI(
    model_name="gpt-4",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    temperature=0,
    verbose=True,
)

/home/kofi/repos/JobGPT/.venv/lib/python3.9/site-packages/langchain/llms/openai.py:171: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/kofi/repos/JobGPT/.venv/lib/python3.9/site-packages/langchain/llms/openai.py:716: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [6]:
# read text from file
with open("../data/job_descriptions/jd_1.txt", "r") as f:
    job_description = f.read()

In [81]:
# template_read = """
# You are given a text of resume, extract the key fields from the resume and output them in a JSON format.
# Make sure to EXCLUDE the fileds such as personal information, contact information, and times.

# Resume Text: {resume_text}
# Answer:
# """
# prompt_read = PromptTemplate(input_variables=["resume_text"], template=template_read)
# chain_read = LLMChain(llm=openai, prompt=prompt_read)
# resume = count_tokens(chain_read, {"resume_text": resume_text})
# print(resume["token_count"])
# resume = json.loads(resume["result"])

In [57]:
# from langchain.output_parsers import ResponseSchema
# from langchain.output_parsers import StructuredOutputParser
# evaluation_schema = ResponseSchema(name="evaluation", description="The overall evaluation of the resume")
# suggestions_schema = ResponseSchema(name="suggestions", description="Suggestions for the resume, format into list")
# revised_resume_schema = ResponseSchema(name="revised_resume", description="The revised resume")
# response_schemas = [evaluation_schema, 
#                     suggestions_schema,
#                     revised_resume_schema]
# output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# format_instructions = output_parser.get_format_instructions()
# print(format_instructions)


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"evaluation": string  // The overall evaluation of the resume
	"suggestions": string  // Suggestions for the resume, format into list
	"revised_resume": string  // The revised resume
}
```


In [10]:
parser = PydanticOutputParser(pydantic_object=ResumeAnalysis)

In [11]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"evaluation": {"title": "Evaluation", "description": "The overall evaluation of the resume", "type": "string"}, "suggestions": {"title": "Suggestions", "description": "Suggestions for the resume, format into list", "type": "array", "items": {"type": "string"}}, "revised_resume": {"title": "Revised Resume", "description": "The revised resume", "type": "string"}}, "required": ["evaluation", "suggestions", "revised_resume"]}
```


### Strategy 1

In [41]:
resume_text = ResumeReader().read("../data/resumes/resume_4.pdf")

In [38]:
system_template = """
You are an experienced career consultalt who helps clients to improve their resumes.
When you are asked to provide evaluation or suggestion, make sure your are critical and specific.
Focus on the use of professional language and the relevancy to the job description.
REMEMBER DO NOT make things up or create fake experiences. 

You response should be in JSON format with following keys: "evaluation", "suggestions", "revised_resume".
"""
user_teamplate = """
You are an experienced career consultalt helping clients with their resumes.
Let's think step by step and section by secion.
First, give an CRITICAL evaluation of the resume focusing on use of professional language and the relevancy to the job description.
Second, provide suggestions on how the client can improve the section. Mention the exact wording used and how the client can reword it. 
Try to list all the problems and give specific suggestions.
Last, give a revision of the work experience section besed on your suggestions.

resume: {resume_text}
Job Description: {job_description}
"""
system_prompt = SystemMessagePromptTemplate.from_template(system_template.strip())
user_prompt = HumanMessagePromptTemplate.from_template(user_teamplate.strip())
resume_analyzer_prompt_1 = ChatPromptTemplate(input_variables=["resume_text", "job_description"], messages=[system_prompt, user_prompt])
chain_analyze_1 = LLMChain(llm=openai, prompt=resume_analyzer_prompt_1)

In [39]:
output = count_tokens(chain_analyze_1, {"resume_text": resume_text, "job_description": job_description})
print(output["token_count"])

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "resume_text": "Jingxuan (Anna) He  \nj255he@uwaterloo.ca | (226)-899-2596 | https://www.linkedin.com/in/anna-he-8297261aa/  \n  \nSUMMARY OF QUALIFICATIONS \n\u2022 \nData-driven candidate with a strong passion for quantitative finance and capital markets \n\u2022 \nExperience in SQL, VBA, Python, C, R, MATLAB with strong exposure to statistical modelling and risk management  \n\u2022 \nExcellent verbal and written communication skills demonstrated by collaborating with different stakeholders \n\u2022 \nStrong problem-solving and time management skills acquired by managing and prioritizing tasks under pressure \nPROFESSIONAL WORK EXPERIENCE \nOPTrust - Quantitative Research Intern, Total Fund Completion Portfolio Strategies                         Sept. 2022 \u2013 Apr. 2023               \n\u2022 \nBuilt backtesting framework in Python including return, volatility, drawdown, and information coefficient to help \ntest

In [40]:
print("===================output======================")
print(output['result'])
output_json = json.loads(output['result'])
print("===================resume======================")
print(output_json["revised_resume"])

===================output======================
{
  "evaluation": "The resume demonstrates a strong background in quantitative finance and capital markets, with relevant experience in SQL, VBA, Python, C, R, and MATLAB. However, the work experience section could be improved by focusing more on credit risk and financial analysis, as mentioned in the job description. Additionally, the resume could benefit from highlighting experience with S&P, Capital IQ, and Moody's, as well as any industry experience in natural resources and building materials.",

  "suggestions": [
    "1. Emphasize credit risk and financial analysis experience in the work experience section.",
    "2. Include any experience with S&P, Capital IQ, and Moody's, if applicable.",
    "3. Mention any industry experience in natural resources and building materials, if applicable.",
    "4. Consider adding a 'Skills' section to highlight specific software and tools mentioned in the job description."
  ],

  "revised_resume":

### Strategy 2

In [42]:
resume_work_experience = """
Credit AnalystVine Group, Toronto ON/BNB Financial Group (Merger)                                        Oct 2020 -Present•Manage approval pipeline through daily review and regular follow-up to collect outstanding documentation to ensure mortgages are funding on time•Review documentation for completeness in highest credit quality standards, with compliance to lender policies and procedures•Provide smooth customer service experience in integration with clients and agent team by managing and optimizing interactions•Establish and maintain positive business relationship with external business partners Head Customer Service RepresentativeCIBCMasonville Banking Centre,London,ON                                                    Oct 2018 - Oct 2020•Consistently provided improving customer experience while ensuring error-free transactions in fast pace environment•Assisted CSR team with reconciling and balancing and efficiently correctingerrors•Supported CSR team in balancing system update, proactively created chit-sheet to avoid balancing errors for team•Successfully achieved quality introduction results at 198% over target referrals on SmartReport in 2019•Proactively identified opportunities to introduce client to FSR team to develop financial plansTeaching Assistant     Department of Economics and Finance, UniversityofGuelph                           Sept 2017 - April 2018•Assisted with grading, conducted tutorial sessions and scheduled office hours with students to address course relatedquestions•Worked with minimal supervision while setting priorities and adapting to multiple demands efficiently•Answered students’ questions in a timely manner
"""

In [47]:
system_template = """
You are an experienced career consultalt who helps clients to improve their resumes.
When you are asked to provide evaluation or suggestion, make sure your are critical and specific.
Focus on the use of professional language and the relevancy to the job description.
REMEMBER DO NOT make things up or create fake experiences. 

You response should be in JSON format with following keys: "evaluation", "suggestions", "revised_resume".
"""
user_teamplate = """
You are a experienced career consultalt helping clients with the {section} section of their resumes given a job description that the client is applying for.
Let's think step by step and experience by experience.
First, give an CRITICAL evaluation of the {section} focusing on use of professional language and the relevancy to the job description.
Second, provide suggestions on how the client can improve the section. Mention the exact wording used and how the client can reword it. 
Try to find all the problems and give specific suggestions.
Last, give a revision of the {section} section besed on your suggestions.

{section}: {section_text}
Job Description: {job_description}
"""
system_prompt = SystemMessagePromptTemplate.from_template(system_template.strip())
user_prompt = HumanMessagePromptTemplate.from_template(user_teamplate.strip())
resume_analyzer_prompt_2 = ChatPromptTemplate(input_variables=["section", "section_text", "job_description"], messages=[system_prompt, user_prompt])
chain_analyze_2 = LLMChain(llm=openai, prompt=resume_analyzer_prompt_2)

In [48]:
output = count_tokens(chain_analyze_2, {"section": "work experience", "section_text": resume_work_experience, "job_description": job_description})
print(output["token_count"])

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "section": "work experience",
  "section_text": "\nCredit AnalystVine Group, Toronto ON/BNB Financial Group (Merger)                                        Oct 2020 -Present\u2022Manage approval pipeline through daily review and regular follow-up to collect outstanding documentation to ensure mortgages are funding on time\u2022Review documentation for completeness in highest credit quality standards, with compliance to lender policies and procedures\u2022Provide smooth customer service experience in integration with clients and agent team by managing and optimizing interactions\u2022Establish and maintain positive business relationship with external business partners Head Customer Service RepresentativeCIBCMasonville Banking Centre,London,ON                                                    Oct 2018 - Oct 2020\u2022Consistently provided improving customer experience while ensuring error-free transactions in fast pace 

In [46]:
print("===================output======================")
print(output['result'])
output_json = json.loads(output['result'])
print("===================resume======================")
print(output_json["revised_resume"])

===================output======================
{
  "evaluation": "The work experience section of the resume demonstrates a background in credit analysis and customer service, which is relevant to the job description. However, the language used in the resume could be more professional and concise. The resume also lacks emphasis on credit risk and financial analysis experience, which are key requirements for the job description.",

  "suggestions": [
    "Reword the Credit Analyst experience to emphasize credit risk and financial analysis skills.",
    "Use more professional language and concise phrasing.",
    "Highlight any experience with S&P, Capital IQ, Moody's, or natural resources and building materials industry.",
    "Mention any relevant academic background or professional designations."
  ],

  "revised_resume": "Work Experience:\n\nCredit Analyst\nVine Group, Toronto ON/BNB Financial Group (Merger)\nOct 2020 - Present\n• Manage credit approval pipeline by reviewing and follo

### Strategy 3

In [57]:
system_template = """
You are an experienced career consultalt who helps clients to improve their resumes.
When you are asked to provide evaluation or suggestion, make sure your are critical and specific.
Focus on the use of professional language and the relevancy to the job description.
REMEMBER DO NOT make things up or create fake experiences. 
"""
evaluation_teamplate = """
You are a experienced career consultalt helping clients with the {section} section of their resumes given a job description that the client is applying for.
Give an CRITICAL evaluation of the section focusing on use of professional language and the relevancy to the job description.

{section}: {section_text}
Job Description: {job_description}
"""
suggestion_teamplate = """
You are a experienced career consultalt helping clients with the {section} section of their resumes given a job description that the client is applying for.
You are also given some high level evalutions. Based on the evaluation, provide suggestions on how the client can improve the section. 
Mention the exact wording used and how the client can reword it. 
Try to find all the problems and give specific suggestions.

{section}: {section_text}
Job Description: {job_description}
Evaluate: {evaluation}
"""
revision_teamplate = """
You are a experienced career consultalt helping clients with the {section} section of their resumes given a job description that the client is applying for.
You are also given some suggestions to the resume.Based on the the suggestions, give a revision of the section besed on your suggestions.

{section}: {section_text}
Job Description: {job_description}
Evaluate: {suggestion}
"""
system_prompt = PromptTemplate.from_template(system_template.strip())
evaluation_prompt = PromptTemplate.from_template(evaluation_teamplate.strip())
suggestion_prompt = PromptTemplate.from_template(suggestion_teamplate.strip())
revision_prompt = PromptTemplate.from_template(revision_teamplate.strip())
chain_analyze_3_system = LLMChain(llm=openai, prompt=system_prompt)
chain_analyze_3_evaluation = LLMChain(llm=openai, prompt=evaluation_prompt, output_key="evaluation")
chain_analyze_3_suggestion = LLMChain(llm=openai, prompt=suggestion_prompt, output_key="suggestion")
chain_analyze_3_revision = LLMChain(llm=openai, prompt=revision_prompt, output_key="revised_resume")
chain_analyze_3 = SequentialChain(
    chains=[chain_analyze_3_system, chain_analyze_3_evaluation, chain_analyze_3_suggestion, chain_analyze_3_revision],
    input_variables=["section", "section_text", "job_description"],
    output_variables=["evaluation", "suggestion", "revised_resume"],
    verbose=True
    )

In [59]:
output = chain_analyze_3({"section": "work experience", "section_text": resume_work_experience, "job_description": job_description})

[chain/start] [1:chain:SequentialChain] Entering Chain run with input:
{
  "section": "work experience",
  "section_text": "\nCredit AnalystVine Group, Toronto ON/BNB Financial Group (Merger)                                        Oct 2020 -Present\u2022Manage approval pipeline through daily review and regular follow-up to collect outstanding documentation to ensure mortgages are funding on time\u2022Review documentation for completeness in highest credit quality standards, with compliance to lender policies and procedures\u2022Provide smooth customer service experience in integration with clients and agent team by managing and optimizing interactions\u2022Establish and maintain positive business relationship with external business partners Head Customer Service RepresentativeCIBCMasonville Banking Centre,London,ON                                                    Oct 2018 - Oct 2020\u2022Consistently provided improving customer experience while ensuring error-free transactions in fas

In [62]:
output

{'section': 'work experience',
 'section_text': '\nCredit AnalystVine Group, Toronto ON/BNB Financial Group (Merger)                                        Oct 2020 -Present•Manage approval pipeline through daily review and regular follow-up to collect outstanding documentation to ensure mortgages are funding on time•Review documentation for completeness in highest credit quality standards, with compliance to lender policies and procedures•Provide smooth customer service experience in integration with clients and agent team by managing and optimizing interactions•Establish and maintain positive business relationship with external business partners Head Customer Service RepresentativeCIBCMasonville Banking Centre,London,ON                                                    Oct 2018 - Oct 2020•Consistently provided improving customer experience while ensuring error-free transactions in fast pace environment•Assisted CSR team with reconciling and balancing and efficiently correctingerrors

In [63]:
print("===================output======================")
print(output)
print("===================resume======================")
print(output["revised_resume"])

===================output======================
{'section': 'work experience', 'section_text': '\nCredit AnalystVine Group, Toronto ON/BNB Financial Group (Merger)                                        Oct 2020 -Present•Manage approval pipeline through daily review and regular follow-up to collect outstanding documentation to ensure mortgages are funding on time•Review documentation for completeness in highest credit quality standards, with compliance to lender policies and procedures•Provide smooth customer service experience in integration with clients and agent team by managing and optimizing interactions•Establish and maintain positive business relationship with external business partners Head Customer Service RepresentativeCIBCMasonville Banking Centre,London,ON                                                    Oct 2018 - Oct 2020•Consistently provided improving customer experience while ensuring error-free transactions in fast pace environment•Assisted CSR team with reconciling